In [1]:
import argparse
import torch
from accelerate import Accelerator, DeepSpeedPlugin
from accelerate import DistributedDataParallelKwargs
from torch import optim
from torch.optim import lr_scheduler

from data_provider.m4 import M4Meta
from models import Autoformer, DLinear
from models import TimeLLM_vertex as TimeLLM

from data_provider.data_factory import data_provider
import time
import random
import numpy as np
import pandas as pd

from utils.losses import smape_loss
from utils.m4_summary import M4Summary
import os

os.environ['CURL_CA_BUNDLE'] = ''
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

from utils.tools import del_files, EarlyStopping, adjust_learning_rate, load_content, test_MS

from data_provider.ean_global_channel import import_true_promo, import_all, check_saved_standardization_data, delete_saved_standardization_data
from google.cloud import bigquery

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(2.0 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))) * 100


fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

/home/user/miniconda/envs/vertex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def test_MS(args, accelerator, model, train_loader, vali_loader, criterion):
    x, _ = train_loader.dataset.last_insample_window()
    y = vali_loader.dataset.timeseries
    x = torch.tensor(x, dtype=torch.float32).to(accelerator.device)
    print("Shape of X eval", x.shape)
    model.eval()
    with torch.no_grad():
        B, _, C = x.shape
        dec_inp = torch.zeros((B, args.pred_len, C)).float().to(accelerator.device)
        dec_inp = torch.cat([x[:, -args.label_len:, :], dec_inp], dim=1)
        outputs = torch.zeros((B, args.pred_len, C)).float().to(accelerator.device)
        id_list = np.arange(0, B, args.eval_batch_size)
        id_list = np.append(id_list, B)
        for i in range(len(id_list) - 1):
            outputs[id_list[i]:id_list[i + 1], :, :] = model(
                x[id_list[i]:id_list[i + 1]],
                None,
                dec_inp[id_list[i]:id_list[i + 1]],
                None
            )
        accelerator.wait_for_everyone()
        outputs = accelerator.gather_for_metrics(outputs)
        print("Shape of output eval before choosing", outputs.shape)
        f_dim = -1 if args.features == 'MS' else 0
        outputs = outputs[:, -args.pred_len:, f_dim:]
        pred = outputs
        true = torch.from_numpy(np.array(y)).to(accelerator.device)
        true = true[:, -args.pred_len:, f_dim:]
        print("Shape of y eval", true.shape)
        batch_y_mark = torch.ones(true.shape).to(accelerator.device)
        true = accelerator.gather_for_metrics(true)
        batch_y_mark = accelerator.gather_for_metrics(batch_y_mark)

        loss = criterion(None, 0, pred, true, batch_y_mark)

    model.train()
    return loss

In [16]:
from math import sqrt

import torch
import torch.nn as nn

from transformers import LlamaConfig, LlamaModel, LlamaTokenizer, GPT2Config, GPT2Model, GPT2Tokenizer, BertConfig, \
    BertModel, BertTokenizer
from layers.Embed import PatchEmbedding
import transformers
from layers.StandardNorm import Normalize
from vertexai.preview import VertexModel # VertexModel
import vertexai
from utils.tools import del_files, EarlyStopping, adjust_learning_rate, vali, load_content, test_MS


from torch import optim
from torch.optim import lr_scheduler
from tqdm import tqdm


import time
import numpy as np
import os

from accelerate import Accelerator, DeepSpeedPlugin
from accelerate import DistributedDataParallelKwargs

from data_provider.data_factory import data_provider
from utils.losses import smape_loss

transformers.logging.set_verbosity_error()


class FlattenHead(nn.Module):
    def __init__(self, n_vars, nf, target_window, head_dropout=0):
        super().__init__()
        self.n_vars = n_vars
        self.flatten = nn.Flatten(start_dim=-2)
        self.linear = nn.Linear(nf, target_window)
        self.dropout = nn.Dropout(head_dropout)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.dropout(x)
        return x


class Model(nn.Module,VertexModel):

    def __init__(self, configs, patch_len=16, stride=8):
        nn.Module.__init__(self)
        VertexModel.__init__(self)
        self.task_name = configs.task_name
        self.pred_len = configs.pred_len
        self.seq_len = configs.seq_len
        self.d_ff = configs.d_ff
        self.top_k = 5
        self.d_llm = configs.llm_dim
        self.patch_len = configs.patch_len
        self.stride = configs.stride
        self.args = configs

        if configs.llm_model == 'LLAMA':
            # self.llama_config = LlamaConfig.from_pretrained('/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/')
            self.llama_config = LlamaConfig.from_pretrained('huggyllama/llama-7b')
            self.llama_config.num_hidden_layers = configs.llm_layers
            self.llama_config.output_attentions = True
            self.llama_config.output_hidden_states = True
            try:
                self.llm_model = LlamaModel.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=True,
                    config=self.llama_config,
                    # load_in_4bit=True
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = LlamaModel.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.llama_config,
                    # load_in_4bit=True
                )
            try:
                self.tokenizer = LlamaTokenizer.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/tokenizer.model",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = LlamaTokenizer.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/tokenizer.model",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=False
                )
        elif configs.llm_model == 'GPT2':
            self.gpt2_config = GPT2Config.from_pretrained('openai-community/gpt2')

            self.gpt2_config.num_hidden_layers = configs.llm_layers
            self.gpt2_config.output_attentions = True
            self.gpt2_config.output_hidden_states = True
            try:
                self.llm_model = GPT2Model.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=True,
                    config=self.gpt2_config,
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = GPT2Model.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.gpt2_config,
                )

            try:
                self.tokenizer = GPT2Tokenizer.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = GPT2Tokenizer.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=False
                )
        elif configs.llm_model == 'BERT':
            self.bert_config = BertConfig.from_pretrained('google-bert/bert-base-uncased')

            self.bert_config.num_hidden_layers = configs.llm_layers
            self.bert_config.output_attentions = True
            self.bert_config.output_hidden_states = True
            try:
                self.llm_model = BertModel.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=True,
                    config=self.bert_config,
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = BertModel.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.bert_config,
                )

            try:
                self.tokenizer = BertTokenizer.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = BertTokenizer.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=False
                )
        else:
            raise Exception('LLM model is not defined')

        if self.tokenizer.eos_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            pad_token = '[PAD]'
            self.tokenizer.add_special_tokens({'pad_token': pad_token})
            self.tokenizer.pad_token = pad_token

        for param in self.llm_model.parameters():
            param.requires_grad = False

        if configs.prompt_domain:
            self.description = configs.content
        else:
            self.description = 'The Electricity Transformer Temperature (ETT) is a crucial indicator in the electric power long-term deployment.'

        self.dropout = nn.Dropout(configs.dropout)

        self.patch_embedding = PatchEmbedding(
            configs.d_model, self.patch_len, self.stride, configs.dropout)

        self.word_embeddings = self.llm_model.get_input_embeddings().weight
        self.vocab_size = self.word_embeddings.shape[0]
        self.num_tokens = 1000
        self.mapping_layer = nn.Linear(self.vocab_size, self.num_tokens)

        self.reprogramming_layer = ReprogrammingLayer(configs.d_model, configs.n_heads, self.d_ff, self.d_llm)

        self.patch_nums = int((configs.seq_len - self.patch_len) / self.stride + 2)
        self.head_nf = self.d_ff * self.patch_nums

        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.output_projection = FlattenHead(configs.enc_in, self.head_nf, self.pred_len,
                                                 head_dropout=configs.dropout)
        else:
            raise NotImplementedError

        self.normalize_layers = Normalize(configs.enc_in, affine=False)

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
            return dec_out[:, -self.pred_len:, :]
        return None

    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):

        x_enc = self.normalize_layers(x_enc, 'norm')

        B, T, N = x_enc.size()
        x_enc = x_enc.permute(0, 2, 1).contiguous().reshape(B * N, T, 1)

        min_values = torch.min(x_enc, dim=1)[0]
        max_values = torch.max(x_enc, dim=1)[0]
        medians = torch.median(x_enc, dim=1).values
        lags = self.calcute_lags(x_enc)
        trends = x_enc.diff(dim=1).sum(dim=1)

        prompt = []
        for b in range(x_enc.shape[0]):
            min_values_str = str(min_values[b].tolist()[0])
            max_values_str = str(max_values[b].tolist()[0])
            median_values_str = str(medians[b].tolist()[0])
            lags_values_str = str(lags[b].tolist())
            prompt_ = (
                f"<|start_prompt|>Dataset description: {self.description}"
                f"Task description: forecast the next {str(self.pred_len)} steps given the previous {str(self.seq_len)} steps information; "
                "Input statistics: "
                f"min value {min_values_str}, "
                f"max value {max_values_str}, "
                f"median value {median_values_str}, "
                f"the trend of input is {'upward' if trends[b] > 0 else 'downward'}, "
                f"top 5 lags are : {lags_values_str}<|<end_prompt>|>"
            )

            prompt.append(prompt_)

        x_enc = x_enc.reshape(B, N, T).permute(0, 2, 1).contiguous() # B, T, N

        prompt = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).input_ids
        prompt_embeddings = self.llm_model.get_input_embeddings()(prompt.to(x_enc.device))  # (batch, prompt_token, dim)

        source_embeddings = self.mapping_layer(self.word_embeddings.permute(1, 0)).permute(1, 0)

        x_enc = x_enc.permute(0, 2, 1).contiguous() # B N T
        enc_out, n_vars = self.patch_embedding(x_enc.to(torch.bfloat16))
        enc_out = self.reprogramming_layer(enc_out, source_embeddings, source_embeddings)
        llama_enc_out = torch.cat([prompt_embeddings, enc_out], dim=1)
        dec_out = self.llm_model(inputs_embeds=llama_enc_out).last_hidden_state
        dec_out = dec_out[:, :, :self.d_ff]

        dec_out = torch.reshape(
            dec_out, (-1, n_vars, dec_out.shape[-2], dec_out.shape[-1]))
        dec_out = dec_out.permute(0, 1, 3, 2).contiguous()

        dec_out = self.output_projection(dec_out[:, :, :, -self.patch_nums:])
        dec_out = dec_out.permute(0, 2, 1).contiguous()

        dec_out = self.normalize_layers(dec_out, 'denorm')

        return dec_out

    @vertexai.preview.developer.mark.train()
    def train_model(self, train_loader, test_loader, vali_loader, path):
        import torch.multiprocessing as mp
        mp.set_start_method('spawn', force=True)
        ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
        deepspeed_plugin = DeepSpeedPlugin(hf_ds_config='./ds_config_zero2.json')
        accelerator = Accelerator(kwargs_handlers=[ddp_kwargs], deepspeed_plugin=deepspeed_plugin)
        print("Accelerator initialized:", accelerator)

        # ## Load datasets 
        # train_data, train_loader = data_provider(self.args, 'train')
        # vali_data, vali_loader = data_provider(self.args, 'val')
        # test_data, test_loader = data_provider(self.args, 'test')


        

        self.args.content = load_content(self.args)
        time_now = time.time()

        train_steps = len(train_loader)
        early_stopping = EarlyStopping(accelerator=accelerator, patience=self.args.patience)

        model_optim = optim.Adam(self.parameters(), lr=self.args.learning_rate)
        criterion = smape_loss()


        if self.args.lradj == 'COS':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)
        else:
            scheduler = lr_scheduler.OneCycleLR(optimizer=model_optim,
                                                steps_per_epoch=train_steps,
                                                pct_start=self.args.pct_start,
                                                epochs=self.args.train_epochs,
                                                max_lr=self.args.learning_rate)
    

        train_loader, vali_loader, self, model_optim, scheduler = accelerator.prepare(train_loader,
                vali_loader, self, model_optim, scheduler)       

        for epoch in range(self.args.train_epochs):
            # train_data, train_loader = data_provider(self.args, 'train')
            # train_loader = accelerator.prepare(train_loader)
            
            iter_count = 0
            train_loss = []

            self.train()
            epoch_time = time.time()

            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                model_optim.zero_grad()
                batch_x = batch_x.float().to(accelerator.device)

                batch_y = batch_y.float().to(accelerator.device)
                batch_y_mark = batch_y_mark.float().to(accelerator.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float().to(accelerator.device)
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(
                    accelerator.device)

                outputs = self(batch_x, None, dec_inp, None)

                f_dim = -1 if self.args.features == 'MS' else 0
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:]

                batch_y_mark = batch_y_mark[:, -self.args.pred_len:, f_dim:]
                loss = criterion(batch_x, 0, outputs, batch_y, batch_y_mark) # 0 cuz we don't need it

                train_loss.append(loss.item())

                if (i + 1) % 100 == 0:
                    accelerator.print(
                        "\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item())
                    )
                    speed = (time.time() - time_now) / iter_count
                    left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
                    accelerator.print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()

                accelerator.backward(loss)
                model_optim.step()

                if self.args.lradj == 'TST':
                    adjust_learning_rate(accelerator, model_optim, scheduler, epoch + 1, self.args, printout=False)
                    scheduler.step()

            accelerator.print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
            train_loss = np.average(train_loss)
            print('########################################################################')
            vali_loss = test_MS(self.args, accelerator, self, train_loader, vali_loader, criterion)
            test_loss = vali_loss
            accelerator.print(
                "Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
                    epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            early_stopping(vali_loss, self, path)  # model saving
            if early_stopping.early_stop:
                accelerator.print("Early stopping")
                break

            if self.args.lradj != 'TST':
                adjust_learning_rate(accelerator, model_optim, scheduler, epoch + 1, self.args, printout=True)
            else:
                accelerator.print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

        accelerator.wait_for_everyone()
        print("Training is Over")

    
    def calcute_lags(self, x_enc):
        q_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
        k_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
        res = q_fft * torch.conj(k_fft)
        corr = torch.fft.irfft(res, dim=-1)
        mean_value = torch.mean(corr, dim=1)
        _, lags = torch.topk(mean_value, self.top_k, dim=-1)
        return lags



class ReprogrammingLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_keys=None, d_llm=None, attention_dropout=0.1):
        super(ReprogrammingLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)

        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_llm, d_keys * n_heads)
        self.value_projection = nn.Linear(d_llm, d_keys * n_heads)
        self.out_projection = nn.Linear(d_keys * n_heads, d_llm)
        self.n_heads = n_heads
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, target_embedding, source_embedding, value_embedding):
        B, L, _ = target_embedding.shape
        S, _ = source_embedding.shape
        H = self.n_heads

        target_embedding = self.query_projection(target_embedding).view(B, L, H, -1)
        source_embedding = self.key_projection(source_embedding).view(S, H, -1)
        value_embedding = self.value_projection(value_embedding).view(S, H, -1)

        out = self.reprogramming(target_embedding, source_embedding, value_embedding)

        out = out.reshape(B, L, -1)

        return self.out_projection(out)

    def reprogramming(self, target_embedding, source_embedding, value_embedding):
        B, L, H, E = target_embedding.shape

        scale = 1. / sqrt(E)

        scores = torch.einsum("blhe,she->bhls", target_embedding, source_embedding)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        reprogramming_embedding = torch.einsum("bhls,she->blhe", A, value_embedding)

        return reprogramming_embedding


In [11]:
class Args:
    def __init__(self):
        self.task_name = 'short_term_forecast'
        self.is_training = 1
        self.model_id = 'promo_ean_channel'
        self.model_comment = 'EAN_Channel'
        self.model = 'TimeLLM'
        self.seed = 2021  # Assuming the seed is not explicitly set in the script
        self.data = 'promo_ean_channel'
        self.root_path = './dataset/true_promo'
        self.data_path = 'all_product_true_promo_train.csv'
        self.features = 'MS'
        self.target = 'sold_units'
        self.loader = 'modal'  # Assuming this is a default value
        self.freq = 'h'  # Assuming this is a default value
        self.checkpoints = './checkpoints/'  # Assuming this is a default value
        self.seq_len = 13  # Assuming this is a default value
        self.label_len = 1  # Assuming this is a default value
        self.pred_len = 17
        self.seasonal_patterns = 'Monthly'  # Assuming this is a default value
        self.enc_in = 7
        self.dec_in = 7
        self.c_out = 7
        self.d_model = 16
        self.n_heads = 8  # Typically set by your model configuration
        self.e_layers = 2  # Typically set by your model configuration
        self.d_layers = 1  # Typically set by your model configuration
        self.d_ff = 64
        self.moving_avg = 25  # Assuming default if not specified in the script
        self.factor = 3
        self.dropout = 0.1  # Assuming default if not specified
        self.embed = 'timeF'  # Assuming default if not specified
        self.activation = 'gelu'  # Assuming default if not specified
        self.output_attention = False  # Assuming default if not specified
        self.patch_len = 1
        self.stride = 8  # Assuming default if not specified
        self.prompt_domain = 0  # Assuming default if not specified
        self.llm_model = 'GPT2'
        self.llm_dim = 768
        self.num_workers = 10  # Default setting
        self.itr = 1
        self.train_epochs = 2
        self.align_epochs = 10  # Assuming default if not specified
        self.batch_size = 1
        self.eval_batch_size = 1  # Assuming default if not specified
        self.patience = 10  # Assuming default if not specified
        self.learning_rate = 0.001
        self.des = 'Exp'
        self.loss = 'spmae'  # Assuming default if not specified
        self.lradj = 'type1'  # Assuming default if not specified
        self.pct_start = 0.2  # Assuming default if not specified
        self.use_amp = False  # Assuming default based on your environment capabilities
        self.llm_layers = 16
        self.percent = 100  # Assuming default if not specified
        self.zero_percent = 0
        self.interpolation = False
        self.interpolation_method = False
        self.fill_discontinuity = False
        self.month = 11
        self.num_weeks = 100
        self.scale=True
        self.embedding = True
        self.embedding_dimension = 2
        self.keep_non_promo = False
        self.channel = 'Offline'
        

# Instantiate the Args
args = Args()


In [13]:

PROJECT_ID = "itg-bpma-gbl-ww-np"  # @param {type:"string"}
REGION = "europe-west1" 
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}
import vertexai
REMOTE_JOB_NAME = "timeseriesllm"
REMOTE_JOB_BUCKET = f"{BUCKET_URI}/{REMOTE_JOB_NAME}"
##################################################################################################
vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=REMOTE_JOB_BUCKET,
)

##################################################################################################
bq_client = bigquery.Client(
    project=PROJECT_ID,  # GCP project used for running the queries and billing
)

#################################################################################################
print("Looking for number of weeks to take")
_,_,_,pred_len = import_true_promo(
        client=bq_client,
        zero_percent=0,
        month=args.month,
        num_weeks=0,
        channel=args.channel,
        fill_discontinuity=args.fill_discontinuity,
        keep_non_promo=args.keep_non_promo
    )
print("Ended up with ", 4*pred_len)
args.num_weeks=4*pred_len
args.pred_len = pred_len
args.label_len = pred_len
print("Let's Load the Data")
if args.interpolation:
    final_data, train_set, test_set, pred_len = import_all(
        client=bq_client,
        zero_percent=args.zero_percent,
        month=args.month,
        num_weeks=args.num_weeks,
        channel=args.channel,
        fill_discontinuity=args.fill_discontinuity,
        keep_non_promo=args.keep_non_promo,
        interpolation_method=args.interpolation_method
    )
else :
    final_data, train_set, test_set, pred_len = import_true_promo(
        client=bq_client,
        zero_percent=args.zero_percent,
        month=args.month,
        num_weeks=args.num_weeks,
        channel=args.channel,
        fill_discontinuity=args.fill_discontinuity,
        keep_non_promo=args.keep_non_promo
    )


################## 
setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.des)
# Construct the path
base_dir = f"dataset/"
if args.interpolation:
    base_dir += f"interpolation_{args.interpolation_method}/"
else : 
    base_dir += f"true_promo/"

base_dir+= f"{args.channel}Channel_Month{args.month}_{args.num_weeks}Weeks"
if args.fill_discontinuity:
    base_dir += "_filldiscont"
if args.keep_non_promo:
    base_dir += "_keepnonpromo"
if args.scale:
    base_dir+="_scaled"
if args.embedding:
    base_dir+=f"_embedding_{args.embedding_dimension}"
base_dir += '/'+setting
os.makedirs(base_dir, exist_ok=True)

train_path = os.path.join(base_dir, "train.csv")
test_path = os.path.join(base_dir, "test.csv")


train_set.to_csv(train_path, index=False)
test_set.to_csv(test_path, index=False)

print(f"Train set saved to: {train_path}")
print(f"Test set saved to: {test_path}")
if args.scale:
     
    args.scale_path = 'scale_path/' + base_dir[8:]
    if check_saved_standardization_data(args.scale_path):
        delete_saved_standardization_data(args.scale_path)


########################################################### configuration ####################
args.pred_len = pred_len
args.label_len = args.pred_len
args.seq_len = int(2*args.pred_len)
args.root_path = base_dir
args.data_path = 'train.csv'
##############################################################################################

Looking for number of weeks to take
number of products before preprocessing 1510
How many ean_global_channel_type: 1510
final data product (if changed we remove discontinuity) 1429
prediction length: 17
Ended up with  68
Let's Load the Data
number of products before preprocessing 1510
How many ean_global_channel_type: 1332
final data product (if changed we remove discontinuity) 1314
prediction length: 17
Train set saved to: dataset/true_promo/OfflineChannel_Month11_68Weeks_scaled_embedding_2/short_term_forecast_promo_ean_channel_TimeLLM_promo_ean_channel_ftMS_sl13_ll17_pl17_dm16_nh8_el2_dl1_df64_fc3_ebtimeF_Exp/train.csv
Test set saved to: dataset/true_promo/OfflineChannel_Month11_68Weeks_scaled_embedding_2/short_term_forecast_promo_ean_channel_TimeLLM_promo_ean_channel_ftMS_sl13_ll17_pl17_dm16_nh8_el2_dl1_df64_fc3_ebtimeF_Exp/test.csv


In [20]:

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.des, ii)

    

    

 
    model = Model(args).float()

    path = os.path.join(args.checkpoints,
                        base_dir[8:] + '_' + str(ii) + '-' + args.model_comment)  # unique checkpoint saving path
    args.content = load_content(args)
    if not os.path.exists(path) :# and accelerator.is_local_main_process:
        os.makedirs(path)


    train_data, train_loader = data_provider(args, 'train')
    test_data, test_loader = data_provider(args, 'test')

    vertexai.preview.init(remote=True)
    #model.train_model.vertex.remote_config.machine_type = "n1-highmen-4"
    model.train_model.vertex.remote_config.container_uri = "europe-west1-docker.pkg.dev/itg-bpma-gbl-ww-np/timeseriesforecasting/torch-train:latest"
    model.train_model.vertex.remote_config.enable_cuda = True
    model.train_model.vertex.remote_config.accelerator_count = 4
    model.train_model(train_loader, test_loader, test_loader,path)
    #torch.save(model.state_dict(), path + '/' + 'checkpoint_v_test1')

scaling the data of train
standarization is over of train
scaling the data of test
standarization is over of test
Remote job created. View the job: https://console.cloud.google.com/ai/platform/locations/europe-west1/training/6060668208682106880?project=238069609727
Start remote execution on Vertex...
gcs_path /gcs/your-bucket-name-itg-bpma-gbl-ww-np-unique/timeseriesllm/remote-job-2024-06-24-13-50-00-dd11e/input/test_loader not found in the metadata. Make sure global serialization metadata is loaded.
W0624 13:54:49.946648 140527512155968 any_serializer.py:471] gcs_path /gcs/your-bucket-name-itg-bpma-gbl-ww-np-unique/timeseriesllm/remote-job-2024-06-24-13-50-00-dd11e/input/test_loader not found in the metadata. Make sure global serialization metadata is loaded.
W0624 13:54:50.469594 140527512155968 any_serializer.py:533] the gcs_path /gcs/your-bucket-name-itg-bpma-gbl-ww-np-unique/timeseriesllm/remote-job-2024-06-24-13-50-00-dd11e/input/test_loader doesn't exist in the metadata. Please 

In [10]:
for name , param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Size: {param.size()} | Values : {param[:1]} \n")

Layer: patch_embedding.value_embedding.tokenConv.weight | Size: torch.Size([32, 1, 3]) | Values : tensor([[[-0.8107,  0.2070, -0.7467]]], grad_fn=<SliceBackward0>) 

Layer: mapping_layer.weight | Size: torch.Size([1000, 50257]) | Values : tensor([[-0.0028,  0.0010,  0.0008,  ...,  0.0027,  0.0015,  0.0031]],
       grad_fn=<SliceBackward0>) 

Layer: mapping_layer.bias | Size: torch.Size([1000]) | Values : tensor([0.0002], grad_fn=<SliceBackward0>) 

Layer: reprogramming_layer.query_projection.weight | Size: torch.Size([1024, 32]) | Values : tensor([[ 0.1551,  0.1534, -0.0059, -0.0869,  0.0856,  0.1022, -0.0441, -0.0434,
          0.0210, -0.1227,  0.0563,  0.0326, -0.0681, -0.0284,  0.0284, -0.0439,
         -0.1313, -0.0647,  0.1476,  0.0233, -0.0952,  0.0208,  0.0993,  0.0400,
          0.0304, -0.1327,  0.1055,  0.0106, -0.1602,  0.0445, -0.1622, -0.0406]],
       grad_fn=<SliceBackward0>) 

Layer: reprogramming_layer.query_projection.bias | Size: torch.Size([1024]) | Values : tenso